In [1]:
import pandas as pd
import requests
import zipfile
import xml.etree.ElementTree as ET
import json
import io
import os
import time
import OpenDartReader
import yaml
from bs4 import BeautifulSoup
import re

from datetime import datetime, timedelta
from pykrx import stock

today = datetime.today().strftime('%Y%m%d')

past_date = datetime.today() - timedelta(days=90)
past_date = past_date.strftime('%Y%m%d')

stock_name = '삼성전자'
ticker = '005930'

In [2]:
# past_date

'20241109'

In [3]:
def get_stock_prices(past_date, today, ticker):
    return stock.get_market_ohlcv(past_date, today, ticker)

In [4]:
# 현재 파일의 디렉토리 경로 가져오기
# current_dir = os.path.dirname(os.path.abspath(__file__))
current_dir = './'
# yaml 파일의 절대 경로 생성
yaml_path = os.path.join(current_dir, '../config/api_keys.yaml')

# YAML 파일 읽기
with open(yaml_path, "r") as file:
    config = yaml.safe_load(file)

# 필요한 값 가져오기
api_key = config['api_keys']['open_dart']
dart = OpenDartReader(api_key)

corp_codes = pd.read_csv('./chatbot_modules/corp_code/corp_code.csv')

In [5]:
def cleanse_text(response):
    '''
    과도한 줄바꿈을 클렌징해주는 코드 (줄바꿈이 연속될 시, 줄바꿈을 1개로 제한)
    '''
    try:
        # 연속된 줄바꿈을 하나의 줄바꿈으로 바꾸기
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text()
        cleaned_text = re.sub(r'\n+', '\n', text)

        print('텍스트 클렌징 성공')
        return cleaned_text
    except Exception as e:
        print(f'텍스트 클렌징 실패 {str(e)}')
        return None

def change_stock_code(corp_codes, stock_code):
    '''
    수집한 정보에 대한 df를 바탕으로 종목코드와 corpcode 변환
    '''
    df = corp_codes[corp_codes['stock_code'] == stock_code]
    return str(df.iloc[0]['corp_code']).zfill(8) if not df.empty else None

def opendart_finance(api_key, corp_code, bsns_year, reprt_code, fs_div):
    '''
    dart로 정보 요청 (api 호출)
    '''
    try:
        url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json'
        params = {
            'crtfc_key': api_key, 
            'corp_code': corp_code,
            'bsns_year': bsns_year,
            'reprt_code': reprt_code,
            'fs_div': fs_div
        }
        r = requests.get(url, params=params)
        jo = json.loads(r.text)
        if jo['status'] == '013':
            return pd.DataFrame()
        print('재무제표 정보 api 요청 성공')
        return pd.json_normalize(jo, 'list')
    except Exception as e:
        print(f'재무제표 정보 api 요청 실패 {str(e)}')
        return None
       
def get_report_code_dict(stock_code: str, target_business_year: str):
    '''
    보고서의 코드를 가져오는 코드. {리포트 이름: 리포트 코드} 형식으로 반환
    '''
    start_year = target_business_year  # 이미 문자열로 받아옴
    end_year = str(int(target_business_year) + 1)

    try:
        report = dart.list(stock_code, start=start_year, end=end_year, kind='A', final=False)

        report_code_dict = {}
        for ix, row in report.iterrows():
            report_name = row['report_nm']
            report_code = row['rcept_no']
            report_code_dict[report_name] = report_code

        print('보고서 코드 다운로드 성공.')
        return report_code_dict
    except Exception as e:
        print(f'보고서 코드 다운로드 실패. {str(e)}')
        return None
     
def get_sub_report_text(url):
    '''
    텍스트를 실제로 api에 요청하는 코드
    최초에 time.sleep(0.1)로 했더니 서버가 문 닫아버림..ㅠ
    일단 time.sleep(5)로 하니까 잘 돌아감 (2024.08.22)
    '''
    try:
        time.sleep(1)
        response = requests.get(url)
        cleaned_text = cleanse_text(response)

        print('보고서 텍스트 api 요청 성공')
        return cleaned_text
    except Exception as e:
        print(f'보고서 텍스트 api 요청 실패 {str(e)}')
        return None
    
def get_sub_report(stock_code:str, target_business_year:str):
    '''
    하위 보고서를 모두 가져오는 main함수.
    하위 보고서의 코드를 get_report_code_dict()를 통해 가져온 다음,
    report_code에 대해 존재하는 모든 하위 문서를 dart.sub_docs()로 추출한 뒤
    get_sub_report_text()로 데이터 클렌징 및 딕셔너리 양식으로 정리하여
    sub_report_dict에 저장
    '''

    try:
        report_code_dict_raw = get_report_code_dict(stock_code, target_business_year)
        keys = list(report_code_dict_raw.keys())[:2]
        report_code_dict = {key: report_code_dict_raw[key] for key in keys}

        report_name_list = list(report_code_dict.keys())
        sub_report_df = dart.sub_docs(report_code_dict[report_name_list[0]])
        sub_report_url = sub_report_df[sub_report_df['title']=='1. 요약재무정보']['url'].item()

        sub_report_text = get_sub_report_text(sub_report_url)
        return sub_report_text
    
    except Exception as e:
        print(f'하위 보고서 다운로드에 실패했습니다. {str(e)}')
        return None

def get_financial_statements(stock_code: str, target_business_year: str, corp_codes):
    '''
    재무제표 데이터를 api로 받아오는 기능의 main 함수
    종목코드와 목표 영업연도 입력 시 df 반환
    '''
    try:
        corp_code = change_stock_code(corp_codes, stock_code)
        reprt_codes = ['11013', '11012', '11014', '11011']
        fs_div = 'CFS'
        
        total_df = pd.DataFrame()

        start_year = int(target_business_year)
        end_year = start_year + 1

        for bsns_year in range(start_year, end_year):
            bsns_year = str(bsns_year)
            for reprt_code in reprt_codes:
                df = opendart_finance(api_key, corp_code, bsns_year, reprt_code, fs_div)
                if not df.empty:
                    df['연결구분'] = '연결' if fs_div == 'CFS' else '개별'
                    total_df = pd.concat([total_df, df], ignore_index=True)
        
        print('재무제표 데이터 다운로드에 성공했습니다.')
        return total_df
    except Exception as e:
        print(f'재무제표 데이터 다운로드에 실패했습니다. {str(e)}')
        return None

In [6]:
fin_statement = get_financial_statements(ticker, past_date[:4], corp_codes)
fin_report = get_sub_report(ticker, past_date[:4])

재무제표 정보 api 요청 성공
재무제표 정보 api 요청 성공
재무제표 정보 api 요청 성공
재무제표 데이터 다운로드에 성공했습니다.
보고서 코드 다운로드 성공.
텍스트 클렌징 성공
보고서 텍스트 api 요청 성공


In [81]:
import os
import json
import openai
import yaml

def get_api_key():
    current_dir = './'
    yaml_path = os.path.join(current_dir, '../config/api_keys.yaml')

    with open(yaml_path, "r") as file:
        config = yaml.safe_load(file)

    api_key = config['api_keys']['open_ai']
    return api_key

# OpenAI API 클라이언트 초기화
client = openai.OpenAI(api_key=get_api_key())

# GPT 호출 함수
def to_GPT(system, prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}
        ],
        max_tokens=3000,
        temperature=0.7
    ).to_dict()
    return response

# JSON 파일 열기 및 읽기
def read_json(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        return {}

def classify_news_type_by_day(json_path):
    target_json = read_json(json_path)

    news_by_tickers = {}
    non_tickers_news = []

    for json in target_json:
        try:
            news_ticker_list = json['ticker']
            for news_ticker in news_ticker_list:
                try:
                    if news_ticker not in news_by_tickers:
                        news_by_tickers[news_ticker] = []
                    news_data_without_ticker = {key: value for key, value in json.items() if key != 'ticker'}
                    news_by_tickers[news_ticker].append(news_data_without_ticker)
                except Exception as e:
                    print(f"Error processing ticker {news_ticker}: {e}")
                    pass
        except KeyError:
            non_tickers_news.append(json)

    return {
        'news_by_tickers': news_by_tickers,
        'non_tickers_news': non_tickers_news
    }

def get_corp_rel_naver_news(ticker):
    news_base_path = './news'
    years = os.listdir(news_base_path)

    final_news_by_tickers = {}
    final_non_tickers_news = {}

    for year in years:
        months_path = os.path.join(news_base_path, year)
        months = os.listdir(months_path)

        final_news_by_tickers[year] = {}
        final_non_tickers_news[year] = {}

        for month in months:
            jsons_list_path = os.path.join(months_path, month)
            jsons_list = os.listdir(jsons_list_path)

            final_news_by_tickers[year][month] = {}
            final_non_tickers_news[year][month] = {}

            for json_path in jsons_list:
                json_path = os.path.join(jsons_list_path, json_path)
                classified_news_dict = classify_news_type_by_day(json_path)

                # 비티커 뉴스 카테고리화
                try:
                    final_non_tickers_news[year][month]['non_tickers_news'] = classified_news_dict['non_tickers_news']
                except:
                    continue

                # 종목별 뉴스 카테고리화
                if ticker in classified_news_dict['news_by_tickers']:
                    final_news_by_tickers[year][month][ticker] = classified_news_dict['news_by_tickers'][ticker]
                else:
                    final_news_by_tickers[year][month][ticker] = f'이 종목에 대한 {json_path}날짜의 기사가 존재하지 않습니다.'
                
                # 비티커 뉴스 카테고리화 (non_tickers_news도 종목별로 정리)
                for news_item in classified_news_dict['non_tickers_news']:
                    # 예시로, 비티커 뉴스의 카테고리화 (키를 'no_ticker'로 설정)
                    final_non_tickers_news[year][month].setdefault('no_ticker', []).append(news_item)
            
    return {
        'final_news_by_tickers': final_news_by_tickers,
        'final_non_tickers_news': final_non_tickers_news
    }

In [83]:
# news_by_tickers_dict = get_corp_rel_naver_news('005930')['final_news_by_tickers']
# non_tickers_news = get_corp_rel_naver_news('005930')['final_non_tickers_news']

In [88]:
summarize_news_system = f"""당신은 증권회사의 리서치센터장입니다.
다음 주어진 월별 뉴스를 보고, 주식 투자에 중요한 내용들을 중심으로 이 내용들을 요약하여 알아보기 쉽게 정리해주세요.
레포트 형식을 따라야 합니다.
"""

In [101]:
import asyncio
import aiohttp
import json

# GPT 호출을 위한 비동기 함수
async def async_to_GPT(session, system, prompt):
    url = 'https://api.openai.com/v1/chat/completions'
    headers = {
        'Authorization': f'Bearer {get_api_key()}',
        'Content-Type': 'application/json'
    }
    data = {
        'model': 'gpt-4o-mini',
        'messages': [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}
        ],
        'max_tokens': 3000,
        'temperature': 0.7
    }
    
    async with session.post(url, headers=headers, json=data) as response:
        response_data = await response.json()
        return response_data['choices'][0]['message']['content']

# 비동기적으로 여러 요청을 처리하는 함수
async def process_news_by_month(news_by_tickers_dict, ticker):
    async with aiohttp.ClientSession() as session:
        tasks = []
        # GPT API 요청을 병렬로 추가
        for year in news_by_tickers_dict:
            for month in news_by_tickers_dict[year]:
                target_news_info = str(news_by_tickers_dict[year][month])
                summarize_ticker_news_prompt = f"다음 종목에 대한 뉴스입니다. {ticker}\n{target_news_info}"
                
                task = asyncio.ensure_future(
                    async_to_GPT(session, summarize_news_system, summarize_ticker_news_prompt)
                )
                tasks.append(task)
        
        # 요청이 모두 끝날 때까지 대기
        responses = await asyncio.gather(*tasks)
        return responses

# 실행 함수
def run_parallel_requests():
    news_by_tickers_dict = get_corp_rel_naver_news('005930')['final_news_by_tickers']
    ticker = '005930'  # 예시로 삼성전자의 티커
    
    # 비동기 함수 호출
    responses = asyncio.run(process_news_by_month(news_by_tickers_dict, ticker))
    
    for idx, response in enumerate(responses):
        print(f"Response {idx + 1}: {response}")
import asyncio
import aiohttp
import json

# GPT 호출을 위한 비동기 함수
async def async_to_GPT(session, system, prompt):
    url = 'https://api.openai.com/v1/chat/completions'
    headers = {
        'Authorization': f'Bearer {get_api_key()}',
        'Content-Type': 'application/json'
    }
    data = {
        'model': 'gpt-4o-mini',
        'messages': [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}
        ],
        'max_tokens': 3000,
        'temperature': 0.7
    }
    
    async with session.post(url, headers=headers, json=data) as response:
        response_data = await response.json()
        return response_data['choices'][0]['message']['content']

# 비동기적으로 여러 요청을 처리하는 함수
async def process_news_by_month(news_by_tickers_dict, ticker):
    async with aiohttp.ClientSession() as session:
        tasks = []
        # GPT API 요청을 병렬로 추가
        for year in news_by_tickers_dict:
            for month in news_by_tickers_dict[year]:
                target_news_info = str(news_by_tickers_dict[year][month])
                summarize_ticker_news_prompt = f"다음 종목에 대한 뉴스입니다. {ticker}\n{target_news_info}"
                
                task = asyncio.ensure_future(
                    async_to_GPT(session, summarize_news_system, summarize_ticker_news_prompt)
                )
                tasks.append(task)
        
        # 요청이 모두 끝날 때까지 대기
        responses = await asyncio.gather(*tasks)
        return responses

# Jupyter 환경에서 asyncio 실행
async def run_parallel_requests():
    news_by_tickers_dict = get_corp_rel_naver_news('005930')['final_news_by_tickers']
    ticker = '005930'  # 예시로 삼성전자의 티커
    
    # 비동기 함수 호출
    responses = await process_news_by_month(news_by_tickers_dict, ticker)
    
    for idx, response in enumerate(responses):
        print(f"Response {idx + 1}: {response}")

# 실행
await run_parallel_requests()  # Jupyter에서는 await를 사용하여 직접 실행

# 실행
run_parallel_requests()

Response 1: # 삼성전자(005930) 뉴스 요약 리포트

## 1. 개요
최근 중국 AI 스타트업 '딥시크'의 AI 모델 'R1' 출시가 국내 및 글로벌 증시에 큰 영향을 미치고 있으며, 특히 삼성전자를 비롯한 반도체 관련 주식들이 큰 하락세를 보이고 있습니다. 이러한 사건은 AI 산업의 비용 구조와 경쟁 구도를 변화시키는 계기가 되고 있습니다.

---

## 2. 주요 뉴스 내용

### 2.1. 딥시크의 등장과 AI 시장 변화
- **딥시크의 특징**: 오픈소스 기반의 AI 모델 'R1' 출시로 인해 AI 개발 비용이 크게 낮아질 것으로 예상됩니다. 기존의 AI 모델들이 수천만 달러의 비용이 드는 것과 달리, 딥시크는 약 558만 달러의 비용으로 개발되었습니다.
- **AI 산업에 미치는 영향**: 딥시크는 기존 AI 모델을 대체하기보다는 새로운 혁신을 촉진할 것으로 전망되고 있습니다. AI 소프트웨어 시장의 활성화가 기대되며, 한국의 AI 관련 기업들이 새로운 성장 동력을 찾을 수 있는 기회가 될 것입니다.

### 2.2. 국내 증시에 미친 영향
- **주가 하락**: 딥시크의 발표 이후, SK하이닉스 주가는 10% 가까이 하락했으며 삼성전자도 2.42% 하락했습니다. 이는 딥시크가 AI 하드웨어 비용을 절감함에 따라 하드웨어 기업들의 수익성에도 부정적인 영향을 미칠 것이라는 우려 때문입니다.
- **환율 상승**: 딥시크의 영향으로 원-달러 환율이 21.4원 상승하여 1452.7원에 거래되었습니다. 이는 미국의 금리 동결과 결합하여 달러 강세를 유도했습니다.

### 2.3. 시장 전망
- **단기적 악재 vs. 중장기적 기회**: 증권가는 딥시크의 출현을 단기적으로는 부정적인 요소로 보지만, 중장기적으로는 AI 시장의 확장을 촉진할 것으로 분석하고 있습니다. AI 소프트웨어 기업들이 더 많은 기회를 가질 것이라는 기대감이 존재합니다.
- **AI 산업의 방향 전환**: 하드웨어 중심에서 소프트웨어 중심으로의 전환이 이루어질 가능성이 있으

<coroutine object run_parallel_requests at 0x15f9168f0>

In [91]:
news_by_tickers_dict_raw = get_corp_rel_naver_news('005930')
news_by_tickers_dict = news_by_tickers_dict_raw['final_news_by_tickers']
non_tickers_news = news_by_tickers_dict_raw['final_non_tickers_news']

non_tickers_monthly_gpt_response = []
years = list(news_by_tickers_dict.keys())
for year in years:
    months = list(news_by_tickers_dict[year].keys())
    for month in months:
        non_tickers_news_info = non_tickers_news[year][month]
        non_tickers_news_response = to_GPT(summarize_news_system, str(non_tickers_news_info))["choices"][0]["message"]["content"]
        non_tickers_monthly_gpt_response.append(non_tickers_news_response)
        print(f"{month} 완료!")

02 완료!


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 128000 tokens. However, your messages resulted in 426253 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [95]:
non_tickers_news_info

{'non_tickers_news': [{'title': '로봇이 택배 배달, AI가 건물 관리…CES도 반한 성수동 미래 오피스',
   'date': '2025.01.28. 오후 5:19',
   'content': '팩토리얼 성수 오피스“테크 기업들이 빌딩 견학을 온다고요?”서울 성수동의 한 오피스 빌딩이 화제다. 1층 로비에선 로봇이 커피를 배달하고, 지하 주차장에선 차량을 옮겨 주차한다.SF영화 한 장면을 연상케 하듯 입주사 직원들이 스마트폰을 이용해 이 모든 것을 제어할 수 있다.대기업과 스타트업들이 줄을 서서 새로운 업무공간을 경험하고 싶어하는 이 곳은 지난해 2월 이지스자산운용이 선보인 ‘팩토리얼 성수’다.23일 금융투자업계에 따르면 지난해 2월 준공한 팩토리얼 성수의 빌딩 운영시스템 ‘탭 & 컨트롤룸’이 CES 2025 스마트시티 부문 혁신상을 수상했다.팩토리얼 성수의 두뇌 역할을 하는 탭 & 컨트롤룸은 스마트시티 부문에서 우수한 평가를 받았다.상업용 오피스 운영체제로는 국내 최초로 실제 현장에서 구현된 사례라는 점에서 기술력을 인정받은 것이다.김현수 이지스자산운용 공간콘텐츠실장은 “올 1월부터는 로봇 택배 배달 서비스도 시작했다”며 “지하 1층 메일룸에 도착한 택배를 로봇이 직접 수령해 사용자의 자리까지 배달한다”고 설명했다.\n\n\n\n 로봇 택배 배달 서비스[사진 출처=이지스자산운용]건물 관리도 지능화되고 있다. CCTV가 촬영한 영상을 AI(인공지능)가 분석해 방문객의 성별, 연령, 동선 데이터를 추출하고 이를 리테일 매장 운영과 건물 관리에 활용하는 시스템을 도입할 예정이다.또 어플리케이션을 통해 접수되는 이용자의 문의사항(VOC)도 AI로 분석해 건물 관리에 활용할 수 있게 할 계획이다.단순한 업무 공간이 아닌 경험을 제공하는 플랫폼으로써 팩토리얼 성수는 인재 확보에 열을 올리는 기업들이 준공 전분터 눈독을 들였고 현재 임대율 100%를 달성한 상태다.지하 1층 워크숍 공간에는 커뮤니티 이벤트와 브랜드 행사를 열 수 있는 문화 공간

In [109]:
import asyncio
import aiohttp
import tiktoken

# GPT 호출을 위한 비동기 함수
async def async_to_GPT(session, system, prompt):
    url = 'https://api.openai.com/v1/chat/completions'
    headers = {
        'Authorization': f'Bearer {get_api_key()}',
        'Content-Type': 'application/json'
    }
    data = {
        'model': 'gpt-4o-mini',
        'messages': [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}
        ],
        'max_tokens': 3000,
        'temperature': 0.7
    }
    
    async with session.post(url, headers=headers, json=data) as response:
        response_data = await response.json()
        return response_data['choices'][0]['message']['content']

# 텍스트를 토큰 수 기준으로 분할하는 함수
def split_text_by_tokens(text, max_tokens=1280000):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    tokens = encoder.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokens[i:i+max_tokens]
        chunks.append(encoder.decode(chunk))
    return chunks

# 비동기적으로 여러 요청을 처리하는 함수
async def process_news_by_month(news_by_tickers_dict, ticker):
    async with aiohttp.ClientSession() as session:
        tasks = []
        # GPT API 요청을 병렬로 추가
        for year in news_by_tickers_dict:
            for month in news_by_tickers_dict[year]:
                non_tickers_news_info = str(news_by_tickers_dict[year][month])
                # 텍스트를 토큰 수 기준으로 분할
                chunks = split_text_by_tokens(non_tickers_news_info)
                
                # 각 분할된 텍스트에 대해 병렬 요청 추가
                for chunk in chunks:
                    summarize_ticker_news_prompt = f"다음 종목에 대한 뉴스입니다. {ticker}\n{chunk}"
                    task = asyncio.ensure_future(
                        async_to_GPT(session, summarize_news_system, summarize_ticker_news_prompt)
                    )
                    tasks.append(task)
        
        # 요청이 모두 끝날 때까지 대기
        responses = await asyncio.gather(*tasks)
        return responses

# Jupyter 환경에서 asyncio 실행
async def run_parallel_requests(ticker):
    news_by_tickers_dict_raw = get_corp_rel_naver_news(ticker)
    news_by_tickers_dict = news_by_tickers_dict_raw['final_news_by_tickers']
    
    # 비동기 함수 호출
    responses = await process_news_by_month(news_by_tickers_dict, ticker)
    
    for idx, response in enumerate(responses):
        print(f"Response {idx + 1}: {response}")

In [110]:
# 실행
await run_parallel_requests(ticker)  # Jupyter에서는 await를 사용하여 직접 실행

KeyError: 'choices'